<a href="https://colab.research.google.com/github/janithcyapa/energy-plus-utility/blob/main/energyplus-utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EnergyPlus Utility

## Installation From Github Repo and Setup Environment

In [46]:
!pip uninstall -y energy-plus-utility

Found existing installation: energy-plus-utility 0.2.1+10
Uninstalling energy-plus-utility-0.2.1+10:
  Successfully uninstalled energy-plus-utility-0.2.1+10


In [1]:
!pip install -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


`prepare_colab_eplus` - Download and Install EnergyPlus 25-1 in `/root/EnergyPlus-25-1-0`

In [2]:
import importlib.metadata
ver = importlib.metadata.version("energy-plus-utility")
print(f"\n✅ Installed 'energy-plus-utility' version: {ver}")



✅ Installed 'energy-plus-utility' version: 0.2.1+12


In [3]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()

# Usage

In [4]:
from eplus.core import EPlusUtil
import types

### Setup Basics

In [5]:
# In your main script or core.py
# verbose, 0 - no logging, 1- verbose logging, 2 - debug logging
OUT_DIR = "/simulation/eplus_out"
sim = EPlusUtil(verbose=2, out_dir=OUT_DIR)

# Reset state before setting model
sim.reset_state()
# Delete previous output directory
sim.delete_out_dir()
# Clear previous outputs
sim.clear_eplus_outputs(patterns="eplusout.*")

Initialized StateMixin
Initialized EnergyPlus State.
Initialized IDFMixin
Initialized LoggingMixin
Initialized SimulationMixin
Initialized UtilsMixin
Initialized HandlersMixin
Initialized SQLMixin
Initialized ControlMixin
EnergyPlus state has been reset.
Output directory does not exist, nothing to delete: /simulation/eplus_out


### Set Model using local files

In [51]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"

# Define the Simulation Model
IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
# Set the Model for Simulation
sim.set_model(IDF, EPW)

EnergyPlus state has been reset.
Model set: IDF='/root/EnergyPlus-25-1-0/ExampleFiles/5ZoneAirCooled.idf', EPW='/root/EnergyPlus-25-1-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw', OUT_DIR='/simulation/eplus_out'


### Set Model From URL

In [6]:
url_idf = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/MultizoneOffice.idf"
url_epw = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/LKA_Colombo-Katunayake.434500_SWERA.epw"

# Set the Model for Simulation
sim.set_model_from_url(url_idf, url_epw)

EnergyPlus state has been reset.
Model set: IDF='/simulation/eplus_out/MultizoneOffice.idf', EPW='/simulation/eplus_out/LKA_Colombo-Katunayake.434500_SWERA.epw', OUT_DIR='/simulation/eplus_out'


### View IDF/API Data

In [58]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False) # Need to run this to get catalog

EnergyPlus state has been reset.


0

In [11]:
sim.api_catalog_df()

{'ACTUATORS':           Kind            ComponentType          ControlType  \
 0     Actuator            Schedule:Year       Schedule Value   
 1     Actuator            Schedule:Year       Schedule Value   
 2     Actuator            Schedule:Year       Schedule Value   
 3     Actuator            Schedule:Year       Schedule Value   
 4     Actuator            Schedule:Year       Schedule Value   
 ...        ...                      ...                  ...   
 1797  Actuator  Outdoor Air System Node  Drybulb Temperature   
 1798  Actuator  Outdoor Air System Node  Wetbulb Temperature   
 1799  Actuator  Outdoor Air System Node           Wind Speed   
 1800  Actuator  Outdoor Air System Node       Wind Direction   
 1801  Actuator              AirLoopHVAC  Availability Status   
 
                    ActuatorKey     Units  
 0     OFFICESMALL BLDG_OCC_SCH       [ ]  
 1     OFFICESMALL ACTIVITY_SCH       [ ]  
 2     WORK EFFICIENCY SCHEDULE       [ ]  
 3            CLOTHING SCHEDU

In [12]:
sim.list_available_variables()

,Kind,VariableName,KeyValue,Units
0,Outputvariable,Lights Electricity Energy,CORE_ZN-OFFICE WHOLEBUILDING - SM OFFICE LIGHTS,J
1,Outputvariable,Lights Electricity Energy,PERIMETER_ZN_1-OFFICE WHOLEBUILDING - SM OFFIC...,J
2,Outputvariable,Lights Electricity Energy,PERIMETER_ZN_2-OFFICE WHOLEBUILDING - SM OFFIC...,J
3,Outputvariable,Lights Electricity Energy,PERIMETER_ZN_3-OFFICE WHOLEBUILDING - SM OFFIC...,J
4,Outputvariable,Lights Electricity Energy,PERIMETER_ZN_4-OFFICE WHOLEBUILDING - SM OFFIC...,J
...,...,...,...,...
70,Outputvariable,Pump Electricity Energy,PUMP VARIABLE SPEED 1,J
71,Outputvariable,Pump Electricity Energy,MAIN SERVICE WATER LOOP WATER MAINS PRESSURE D...,J
72,Outputvariable,Environmental Impact Total N2O Emissions Carbo...,SITE,kg
73,Outputvariable,Environmental Impact Total CH4 Emissions Carbo...,SITE,kg


In [13]:
sim.list_available_meters()

,Kind,MeterName,Units
0,Outputmeter,Electricity:Facility,J
1,Outputmeter,Electricity:Building,J
2,Outputmeter,Electricity:Zone:CORE_ZN ZN,J
3,Outputmeter,Electricity:SpaceType:OFFICE WHOLEBUILDING - S...,J
4,Outputmeter,InteriorLights:Electricity,J
...,...,...,...
133,Outputmeter,General:WaterSystems:Electricity,J
134,Outputmeter,Pumps:Electricity,J
135,Outputmeter,General:Pumps:Electricity,J
136,Outputmeter,Carbon Equivalent:Facility,kg


In [14]:
sim.list_available_actuators()

,Kind,ComponentType,ControlType,ActuatorKey,Units
0,Actuator,Schedule:Year,Schedule Value,OFFICESMALL BLDG_OCC_SCH,[ ]
1,Actuator,Schedule:Year,Schedule Value,OFFICESMALL ACTIVITY_SCH,[ ]
2,Actuator,Schedule:Year,Schedule Value,WORK EFFICIENCY SCHEDULE,[ ]
3,Actuator,Schedule:Year,Schedule Value,CLOTHING SCHEDULE,[ ]
4,Actuator,Schedule:Year,Schedule Value,AIR VELOCITY SCHEDULE,[ ]
...,...,...,...,...,...
1797,Actuator,Outdoor Air System Node,Drybulb Temperature,MODEL OUTDOOR AIR NODE,[C]
1798,Actuator,Outdoor Air System Node,Wetbulb Temperature,MODEL OUTDOOR AIR NODE,[C]
1799,Actuator,Outdoor Air System Node,Wind Speed,MODEL OUTDOOR AIR NODE,[m/s]
1800,Actuator,Outdoor Air System Node,Wind Direction,MODEL OUTDOOR AIR NODE,[degree]


In [ ]:
objects_types = sim.get_idf_object_types()

print(f"{'OBJECT TYPE':<40} | {'COUNT'}")
print("-" * 50)
for obj_type, count in objects_types:
    print(f"{obj_type:<40} | {count}")

OBJECT TYPE                              | COUNT
--------------------------------------------------
No                                       | 152
Schedule:Day:Interval                    | 113
Autosize                                 | 71
Fractional                               | 64
Temperature                              | 49
Pipe:Adiabatic                           | 48
BuildingSurface:Detailed                 | 43
Branch                                   | 42
Schedule:Week:Daily                      | 38
Schedule:Year                            | 38
OnOff                                    | 37
NoSun                                    | 31
NoWind                                   | 31
Surface                                  | 26
FenestrationSurface:Detailed             | 23
Construction                             | 23
Wall                                     | 20
Window                                   | 20
Attic                                    | 19
General                 

In [ ]:
object = sim.extract_idf_objects("Zone")
print(object)

['Attic ZN', 'Core_ZN ZN', 'Perimeter_ZN_1 ZN', 'Perimeter_ZN_2 ZN', 'Perimeter_ZN_3 ZN', 'Perimeter_ZN_4 ZN', '']


### SetupSQL

In [10]:
sim.ensure_output_sqlite()

EnergyPlus state has been reset.


'/simulation/eplus_out/MultizoneOffice__sqlite.idf'

In [11]:
sim.ensure_output_variables([
    {"name": "System Node Temperature", "key": "*", "freq": "Hourly"},
    {"name": "System Node Mass Flow Rate", "key": "*", "freq": "Hourly"},
    {"name": "System Node Humidity Ratio", "key": "*", "freq": "Hourly"},
    {"name": "Zone Air Temperature", "key": "*", "freq": "Hourly"},
    {"name": "Zone Air Relative Humidity", "key": "*", "freq": "Hourly"}
])

sim.ensure_output_variables([
    {"name": "Site Outdoor Air Drybulb Temperature", "key": "Environment"},
    {"name": "Site Outdoor Air Humidity Ratio", "key": "Environment"},
    {"name": "Site Wind Speed", "key": "Environment"},
    {"name": "Site Direct Solar Radiation Rate per Area", "key": "Environment"}
])

EnergyPlus state has been reset.
EnergyPlus state has been reset.


'/simulation/eplus_out/MultizoneOffice__sqlite__vars__vars.idf'

In [12]:
sim.ensure_output_meters([
    "Electricity:Facility",          # Total building electricity
    "Fans:Electricity",              # Electricity used by HVAC fans
    "Cooling:Electricity",           # Electricity used for cooling (Chillers/DX coils)
    "Heating:Electricity",           # Electricity used for heating
    "ElectricityPurchased:Facility"  # Total grid electricity bought
], freq="Hourly")

EnergyPlus state has been reset.


'/simulation/eplus_out/MultizoneOffice__sqlite__vars__vars__meters.idf'

### Run Basic Simualtion

In [ ]:
sim.enable_runtime_logging()
# sim.disable_runtime_logging()

In [19]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False)

EnergyPlus state has been reset.


0

In [ ]:
sim.run_design_day()

In [39]:
sim.run_annual()

Deleted output file: /simulation/eplus_out/eplusout.sql
Deleted output file: /simulation/eplus_out/eplusout.err
Deleted output file: /simulation/eplus_out/eplusout.audit
EnergyPlus state has been reset.


0

### Usage of Handlers

In [ ]:
# --- STEP A: DEFINE THE CUSTOM FUNCTION ---
# Notice we define it with 'self' as the first argument, just like a class method.
def dr_supervisor_logic(self, state):
    """
    A simple logic that checks the time and prints a message.
    """
    # Just printing for this demo, but you would put control logic here
    time = self.exchange.current_time(state)
    print(f"[SUPERVISOR] Checking status at time: {time:.2f}")

# --- STEP B: INJECT IT  ---
# This binds the function ONLY to this 'sim' object.
# It creates a true method where 'self' is passed automatically.
sim.my_supervisor = types.MethodType(dr_supervisor_logic, sim)


In [ ]:
print("--- 1. REGISTERING ---")
# We register the NAME of the attribute we just attached ("my_supervisor")
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)
print(f"Registered methods: {registered}")

print("\n--- 2. LISTING ---")
# Check what is currently scheduled for the 'begin' hook
current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

print("\n--- 3. DISABLING (Pausing) ---")
# Scenario: It's Winter, we don't need Demand Response.
# We disable the hook so the logic stops running, but we don't delete it.
sim.disable_hook("begin")
print("Hook 'begin' is now DISABLED. (Simulation runs, but supervisor sleeps)")

print("\n--- 4. ENABLING (Resuming) ---")
# Scenario: Summer is back. Turn the logic back on.
sim.enable_hook("begin")
print("Hook 'begin' is now ENABLED. (Supervisor is active again)")

print("\n--- 5. UNREGISTERING (Deleting) ---")
# Scenario: We want to remove this logic entirely to replace it or clean up.
remaining = sim.unregister_handlers(
    "begin",
    ["my_supervisor"] # Name to remove
)
print(f"Unregistered 'my_supervisor'. Remaining handlers: {remaining}")

# --- RUN ---
# sim.run_annual()

In [ ]:
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)

current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

# --- RUN ---
sim.run_annual()

### Control

In [15]:
def zone_temp_controller(self, state):
    """
    Simple Bang-Bang or Proportional controller for Zone Temperature.
    """
    # 1. READ: Get the current zone temperature
    # Using your 'runtime_get_variable' helper
    current_temp = self.runtime_get_variable(
        state,
        name="Zone Mean Air Temperature",
        key="CORE_ZN"
    )

    # Safety check: API might not be ready during first few ticks
    if current_temp is None:
        return

    # 2. DECIDE: Logic to determine the new setpoint
    # Example: If it's too hot (> 25C), drop setpoint to 22C. Otherwise, stay at 24C.
    if current_temp > 10.0:
        new_setpoint = 12.0
    else:
        new_setpoint = 14.0

    # 3. WRITE: Update the cooling setpoint schedule
    # Using your 'runtime_set_actuator' helper
    success = self.runtime_set_actuator(
        state,
        component_type="Schedule:Compact",
        control_type="Schedule Value",
        actuator_key="Cooling_Setpoint_Sched",
        value=new_setpoint,
        log=True # This will use your internal _log to record the change
    )

# 4. BIND & REGISTER
# Attach the function to your simulation object
sim.zone_control = types.MethodType(zone_temp_controller, sim)

# Register it to the 'begin_zone_timestep_before_set_point_manager' hook
# This is the best hook for temperature control as it happens right before
# HVAC systems calculate their response for the step.
sim.register_handlers(
    "begin",
    ["zone_control"]
)

sim.run_annual()

Deleted output file: /simulation/eplus_out/eplusout.sql
Deleted output file: /simulation/eplus_out/eplusout.err
Deleted output file: /simulation/eplus_out/eplusout.audit
EnergyPlus state has been reset.


0

### SQL

In [28]:
df_list = sim.list_sql_zone_variables(like="Zone %Temperature%")
print(df_list)

                   Name           KeyValue Units ReportingFrequency  n_rows
0  Zone Air Temperature           ATTIC ZN     C             Hourly    8760
1  Zone Air Temperature         CORE_ZN ZN     C             Hourly    8760
2  Zone Air Temperature  PERIMETER_ZN_1 ZN     C             Hourly    8760
3  Zone Air Temperature  PERIMETER_ZN_2 ZN     C             Hourly    8760
4  Zone Air Temperature  PERIMETER_ZN_3 ZN     C             Hourly    8760
5  Zone Air Temperature  PERIMETER_ZN_4 ZN     C             Hourly    8760


In [30]:
df_list = sim.list_sql_zone_variables(like="System Node %Temperature%")
print(df_list)

                        Name  \
0    System Node Temperature   
1    System Node Temperature   
2    System Node Temperature   
3    System Node Temperature   
4    System Node Temperature   
..                       ...   
229  System Node Temperature   
230  System Node Temperature   
231  System Node Temperature   
232  System Node Temperature   
233  System Node Temperature   

                                              KeyValue Units  \
0    40GAL ELECTRICITY WATER HEATER - 40KBTU/HR 1.0...     C   
1    40GAL ELECTRICITY WATER HEATER - 40KBTU/HR 1.0...     C   
2    40GAL ELECTRICITY WATER HEATER - 40KBTU/HR 1.0...     C   
3    40GAL ELECTRICITY WATER HEATER - 40KBTU/HR 1.0...     C   
4    AIR TERMINAL SINGLE DUCT VAV REHEAT 1 DAMPER O...     C   
..                                                 ...   ...   
229                  PIPE ADIABATIC 3 INLET WATER NODE     C   
230                  PIPE ADIABATIC 4 INLET WATER NODE     C   
231                  PIPE ADIABATIC 4 I

In [16]:
sim.plot_sql_zone_variable(
    var_name="Zone Air Temperature",
    keys=["PERIMETER_ZN_1 ZN", "CORE_ZN ZN"],
    resample="1H",
    start="2002-01-01"
)

/usr/local/lib/python3.12/dist-packages/eplus/components/sql.py:796: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [34]:
sim.plot_sql_meters(
    ["Electricity:Facility", "Fans:Electricity"],
    resample="1D"  # Look at daily totals
)

In [35]:
# Define your "Control" (Setpoints) and "Output" (Actual Temps)
controls = [{"kind": "var", "name": "Zone Thermostat Heating Setpoint Temperature", "key": "*"}]
outputs  = [{"kind": "var", "name": "Zone Air Temperature", "key": "*"}]

# Generate a correlation heatmap
sim.plot_sql_cov_heatmap(
    control_sels=controls,
    output_sels=outputs,
    stat="corr",  # Show correlation (-1 to 1)
    resample="1h"
)

ValueError: No rows matched. Check names/keys/frequencies, or set include_design_days=True.
Tip: reporting_freq=None disables the frequency filter.

In [40]:
# Export outdoor conditions to a CSV for external analysis
sim.export_weather_sql_to_csv(csv_filename="site_weather_data.csv")

[weather→csv] Wrote /simulation/eplus_out/site_weather_data.csv with shape (8760, 5)
Window: 2006-01-01 00:00:00 → 2006-12-31 23:00:00 (rows=8760)


/usr/local/lib/python3.12/dist-packages/eplus/components/sql.py:1659: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



,series,rows,min,mean,max
0,Site Outdoor Air Drybulb Temperature [C],8760,19.166667,27.370731,46.875000
1,Site Outdoor Air Humidity Ratio [kgWater/kgDry...,8760,0.003599,0.018360,0.027129
2,Site Wind Speed [m/s],8760,0.000000,3.194983,62.425000
3,Site Direct Solar Radiation Rate per Area [W/m2],8760,0.000000,152.704481,880.833333


('/simulation/eplus_out/site_weather_data.csv',
                                               series  rows        min  \
 0           Site Outdoor Air Drybulb Temperature [C]  8760  19.166667   
 1  Site Outdoor Air Humidity Ratio [kgWater/kgDry...  8760   0.003599   
 2                              Site Wind Speed [m/s]  8760   0.000000   
 3   Site Direct Solar Radiation Rate per Area [W/m2]  8760   0.000000   
 
          mean         max  
 0   27.370731   46.875000  
 1    0.018360    0.027129  
 2    3.194983   62.425000  
 3  152.704481  880.833333  )

In [42]:
# Get a raw DataFrame of zone temperatures
df = sim.get_sql_series_dataframe([
    {"kind": "var", "name": "Zone Air Temperature", "key": "*"}
])
display(df)
# Now you can use df.mean(), df.groupby(), etc.

,timestamp,trace,value,kind,name,key,units,freq
0,2006-01-01 00:00:00,Zone Air Temperature (ATTIC ZN) [C],28.364456,var,Zone Air Temperature,ATTIC ZN,C,Hourly
35040,2006-01-01 00:00:00,Zone Air Temperature (PERIMETER_ZN_3 ZN) [C],21.545976,var,Zone Air Temperature,PERIMETER_ZN_3 ZN,C,Hourly
26280,2006-01-01 00:00:00,Zone Air Temperature (PERIMETER_ZN_2 ZN) [C],21.739479,var,Zone Air Temperature,PERIMETER_ZN_2 ZN,C,Hourly
17520,2006-01-01 00:00:00,Zone Air Temperature (PERIMETER_ZN_1 ZN) [C],22.529708,var,Zone Air Temperature,PERIMETER_ZN_1 ZN,C,Hourly
8760,2006-01-01 00:00:00,Zone Air Temperature (CORE_ZN ZN) [C],21.373155,var,Zone Air Temperature,CORE_ZN ZN,C,Hourly
...,...,...,...,...,...,...,...,...
35039,2006-12-31 23:00:00,Zone Air Temperature (PERIMETER_ZN_2 ZN) [C],22.748766,var,Zone Air Temperature,PERIMETER_ZN_2 ZN,C,Hourly
17519,2006-12-31 23:00:00,Zone Air Temperature (CORE_ZN ZN) [C],22.598854,var,Zone Air Temperature,CORE_ZN ZN,C,Hourly
8759,2006-12-31 23:00:00,Zone Air Temperature (ATTIC ZN) [C],27.609665,var,Zone Air Temperature,ATTIC ZN,C,Hourly
43799,2006-12-31 23:00:00,Zone Air Temperature (PERIMETER_ZN_3 ZN) [C],22.619834,var,Zone Air Temperature,PERIMETER_ZN_3 ZN,C,Hourly


### Test

In [ ]:
args = [
        '-w', EPW,
        '-d', OUT_DIR,
        IDF
    ]

print("Starting simulation...")
exit_code = sim.runtime.run_energyplus(sim.state, args)

if exit_code == 0:
    print("Simulation success!")
else:
    print("Simulation failed!")

In [ ]:
print(dir(sim))
print(hasattr(sim, 'run_design_day'))